<a href="https://colab.research.google.com/github/navjotsingh151/Sensor-TimeSeriesAnalysis/blob/master/InsuranceData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Economical Insurance Case Study

---

## Index
1. **Ordering Line** : Analysis of Products that were ordered but have not reached to any further stages
2. **Assembly Line Part 1** : Analysis of Assembly line with missing assembly date but reached further supply chain pipeline.
3. **Assembly Line Part 2** : Products that were assembled but were never shipped to upfitment stage or dealer delivery.
4. **Upfitment Line Part 1** : Analysis of upfitment stage with missing upfitment stage but product was delivered.
5. **Upfitment Line Part 2** : Analysis of missing upfitment shipment date but upfitment was completed.
6. **Delivery Line** : Analysis of products that were delivered directly to clients withour dealer.


---



**Note**: Charts are not visible in Github. Open notebook in Colab using above link

In [ ]:
!pip install plotly==4.8.1
!pip install chart_studio
!pip install pyxlsb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot
import warnings
warnings.filterwarnings("ignore")
pd.options.plotting.backend = "plotly"

In [ ]:
data = pd.read_excel('/content/Sample Inventory Report_20200610_n.xlsx', 
                     header=0)
data.head()

,Identifier,Year,OEM,Model,Ordered Date,Assembly Completed Date,Date Shipped to Upfitter,Date Upfit Complete,Ship Date To Dealer,Delivered to Dealer,Delivered to Client
0,ABCDEFGH123456789,2019,MAN E,Model M,2019-05-09,2019-09-26,NaT,2019-10-11,2019-10-23,2019-11-12,2019-11-12
1,ABCDEFGH123456790,2019,MAN E,Model M Cargo,2019-05-09,2019-09-29,NaT,2019-10-11,2019-10-15,2019-10-16,2019-11-06
2,ABCDEFGH123456791,2019,MAN E,Model M,2019-05-09,2019-09-26,NaT,NaT,2019-10-20,2019-10-21,NaT
3,ABCDEFGH123456792,2019,MAN E,Model M,2019-05-09,2019-09-25,NaT,2019-10-08,2019-11-01,2019-11-01,2019-11-18
4,ABCDEFGH123456793,2019,MAN E,Model M,2019-05-09,2019-09-29,NaT,2019-10-14,NaT,2019-11-21,2019-11-21


In [ ]:
## Basic step is to see the datatypes and check variation for numeric data
print("Shape of Data :", data.shape)
data.info()

Shape of Data : (53570, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53570 entries, 0 to 53569
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Identifier                53559 non-null  object        
 1   Year                      53570 non-null  int64         
 2   OEM                       53570 non-null  object        
 3   Model                     53570 non-null  object        
 4   Ordered Date              53570 non-null  datetime64[ns]
 5   Assembly Completed Date   49654 non-null  datetime64[ns]
 6   Date Shipped to Upfitter  18330 non-null  datetime64[ns]
 7   Date Upfit Complete       41406 non-null  datetime64[ns]
 8   Ship Date To Dealer       32118 non-null  datetime64[ns]
 9   Delivered to Dealer       41128 non-null  datetime64[ns]
 10  Delivered to Client       37230 non-null  datetime64[ns]
dtypes: datetime64[ns](7), int64(1), object(3)
memory usa

In [ ]:
## Check for any nulls
data.isnull().sum()
print(data.isna().sum())
# data.drop(['id', 'name'], axis = 1, inplace=True)

Identifier                     11
Year                            0
OEM                             0
Model                           0
Ordered Date                    0
Assembly Completed Date      3916
Date Shipped to Upfitter    35240
Date Upfit Complete         12164
Ship Date To Dealer         21452
Delivered to Dealer         12442
Delivered to Client         16340
dtype: int64


# **1. Ordering Line : Parts that were Ordered but not reached further stages**

In [242]:
## Parts that are ordered and not proceeded further

data_only_ordered  = data[(data['Assembly Completed Date'].isna() ) & ( data['Delivered to Dealer'].isna() ) &
                                                 ( data['Date Shipped to Upfitter'].isna() ) &
                                                 ( data['Delivered to Client'].isna() ) &
                                                 ( data['Date Upfit Complete'].isna() ) &
                                                 ( data['Ship Date To Dealer'].isna()  ) ]
print("Products that are noly ordered but were not moved down the further stages:", len(data_only_ordered))
data_only_ordered.head()                                           

Products that are noly ordered but were not moved down the further stages: 3828


,Identifier,Year,OEM,Model,Ordered Date,Assembly Completed Date,Date Shipped to Upfitter,Date Upfit Complete,Ship Date To Dealer,Delivered to Dealer,Delivered to Client
6364,ABCDEFGH123463458,2019,MAN E,Model M,2019-02-19,NaT,NaT,NaT,NaT,NaT,NaT
13839,ABCDEFGH123471117,2020,MAN C,Model B,2020-01-09,NaT,NaT,NaT,NaT,NaT,NaT
14181,ABCDEFGH123471493,2021,MAN C,Model G,2020-01-13,NaT,NaT,NaT,NaT,NaT,NaT
14182,ABCDEFGH123471494,2021,MAN C,Model G,2020-01-13,NaT,NaT,NaT,NaT,NaT,NaT
14183,ABCDEFGH123471495,2021,MAN C,Model G,2020-01-13,NaT,NaT,NaT,NaT,NaT,NaT


In [243]:
# Distribution of data along the order date

data_only_ordered_c = data_only_ordered.iloc[:, 0:5].copy()

data_only_ordered_c.groupby('Ordered Date').count()['Year'].plot()
fig = data_only_ordered_c.groupby('Ordered Date').count()['Year'].plot(title="Overall Distribution of products that are only ordered but not processed further down the supply chain", 
                                                                       template="seaborn",
                                                                       labels=dict(index="Ordered Date", 
                                                                                   value="Number of Parts",
                                                                                   variable="Option"
                                                                                   ))
fig.update_layout(showlegend=False)
fig.update_traces(mode="markers+lines")
fig.show()

In [244]:
df = data_only_ordered_c.groupby(['Ordered Date', 'OEM'], as_index=False).count()
fig = px.line(df, x="Ordered Date", y="Year", color="OEM",
              title="Parts Distribution based on OEM that are only Ordered but not processed further ", 
              labels= dict(Year = 'Number of Parts'))
fig.update_traces(mode="markers+lines")
fig.update_layout(hovermode="x unified")

fig.show()

In [245]:
df = data_only_ordered_c.groupby('OEM', as_index=False).count()[['OEM','Identifier' ]]
df.rename(columns= {'Identifier': 'Number of parts'}, inplace = True)
df

,OEM,Number of parts
0,MAN B,1
1,MAN C,84
2,MAN E,1
3,MAN F,525
4,MISC,0
5,Man A,3206


## Observation

It is evident that there is some order issue with **Original Equipment Manufacture (OEM)** - **Man A** and **MAN F** as they have highest products that were ordered but not shipped to dealer or client.

1. Man A : **3206 Parts**
2. MAN F : **525 Parts**

#Question:

What is the order collection process for OEM - Man F and Man A? How long is the time taken to process OEM parts to further stages? 


In [ ]:
## Analaysing OEM - Man A
df = data_only_ordered_c[( data_only_ordered_c.OEM == 'Man A' ) | 
                         ( data_only_ordered_c.OEM == 'MAN F' )].groupby(['Ordered Date', 'OEM', 'Model'],
                                                                     as_index=False).count()
# df = df[df.oem == 'Man A']
fig = px.line(df, x="Ordered Date", y="Year", color="Model", 
              facet_col = "OEM",
              title="Parts Distribution with OEM and Model based on order date", 
              labels= dict(Year = 'Number of Parts'))
fig.update_traces(mode="markers+lines")
fig.update_layout(hovermode="x unified")

fig.show()

In [ ]:
df = data_only_ordered_c.groupby(['OEM', 'Model'], as_index=False).count()[['OEM','Model','Identifier' ]]
df.rename(columns= {'Identifier': 'Number of parts'}, inplace = True)
df

,OEM,Model,Number of parts
0,MAN B,Model H,1
1,MAN C,Model B,1
2,MAN C,Model G,83
3,MAN E,Model M,1
4,MAN F,Model J,1
5,MAN F,Model L,524
6,MISC,Model P,0
7,Man A,Model E CUTAWAY,2
8,Man A,Model F COMMERCIAL STRIPPED,1375
9,Man A,Model N CARGO,1829


## Observation

1. It is evident that products are ordered in large quantity on **Dec 4, 2019** and **Dec 13, 2019**. 

2. OEM **Man A** with model **Model F Commercial Stripped** and **Model N Cargo** were the top to models that were ordered in high volumne but were not processed further.

## Question
1. What is average time taken by above two models to process further ?
2. Is ther any min. quantity these two model parts need to be followed ? As the order volumn is quite high


# **2. Assembly Line Part 1 :Without Assembly date but proceeded to further stages**

In [246]:
## Parts those don't have assembly completed dates but moved forward with other stages

data_no_assembly_date  = data[(data['Assembly Completed Date'].isna() ) & (( data['Delivered to Dealer'].notna() ) |
                                                 ( data['Date Shipped to Upfitter'].notna() )|
                                                 ( data['Delivered to Client'].notna() )| 
                                                 ( data['Date Upfit Complete'].notna() ) |
                                                 ( data['Ship Date To Dealer'].notna()  )) ]


print("Products without assembly date but were in next stages:", len(data_no_assembly_date))
data_no_assembly_date.head()

Products without assembly date but were in next stages: 88


,Identifier,Year,OEM,Model,Ordered Date,Assembly Completed Date,Date Shipped to Upfitter,Date Upfit Complete,Ship Date To Dealer,Delivered to Dealer,Delivered to Client
2160,ABCDEFGH123459065,2019,MAN E,Model M,2019-04-04,NaT,NaT,2019-10-29,NaT,2019-10-23,NaT
2250,ABCDEFGH123459158,2019,MAN E,Model M,2019-03-18,NaT,NaT,2020-03-27,2020-02-05,2020-04-15,2020-04-16
9604,ABCDEFGH123466776,2018,MAN E,Model M,2018-07-30,NaT,NaT,NaT,NaT,NaT,2018-10-16
9605,ABCDEFGH123466777,2018,MAN E,Model M,2018-07-30,NaT,NaT,NaT,NaT,NaT,2018-11-16
9965,ABCDEFGH123467140,2018,MAN E,Model M,2018-07-30,NaT,NaT,NaT,NaT,NaT,2018-10-16


## Observation
Total **88 products** have no assembly date but proceede to further stages.
1. **77** Products were delivered to client
2. **7** Products were delivered to dealer but not to client.
3. **4** Products had completed upfit but were not delivered to dealer or client


In [247]:
## Adding some date atributes : Year - Month based on upfit date and dealer deliver date and Client deliver date

data_no_assembly_date['Delivered Month'] = data_no_assembly_date['Delivered to Client'].dt.to_period('M')
data_no_assembly_date['Delivered Dealer Month'] = data_no_assembly_date['Delivered to Dealer'].dt.to_period('M')
data_no_assembly_date['Delivered Month'].fillna(data_no_assembly_date['Delivered Dealer Month'], inplace = True)
data_no_assembly_date['Date Upfit Complete Month'] = data_no_assembly_date['Date Upfit Complete'].dt.to_period('M')
data_no_assembly_date['Delivered Month'].fillna(data_no_assembly_date['Date Upfit Complete Month'], inplace = True)

## Aggregating Data based on OEM , Model, Delivered Month

data_no_assembly_date_c = data_no_assembly_date.groupby(['OEM','Model', 'Delivered Month'], as_index=False).count()[['OEM','Model','Delivered Month','Identifier' ]].copy()
data_no_assembly_date_c.rename(columns= {'Identifier': 'Number of parts'}, inplace = True)
data_no_assembly_date_c['Delivered Month'] = data_no_assembly_date_c['Delivered Month'].astype(str)

#Plotting data
fig = px.line(data_no_assembly_date_c, x="Delivered Month", y="Number of parts", color="Model", 
              facet_col = "OEM",
              title="" 
              )
fig.update_traces(mode="markers+lines")
# fig.update_layout(hovermode="x unified")

fig.show()
print(data_no_assembly_date_c.groupby(['OEM','Model'], 
                                      as_index=False).sum()[['OEM','Model','Number of parts']].sort_values(by = 'Number of parts', ascending=False))


     OEM             Model  Number of parts
2  MAN E           Model M               46
5  Man A           Model N               34
0  MAN B          Model H                 4
1  MAN D           Model I                1
3  MAN F           Model L                1
4  Man A  Model E STRIPPED                1
6  Man A           Model O                1


## Observation

1. Products with **OEM**  - **Man E** and **Man A** have maximum products that don't have assembly date but were delivered to dealer or client.
2. OEM with **Man E** had something wrong in **3rd Quater of 2018** as they have maximum products that don't have assembly date
3. OEM with **Man A** has something wrong around **Aug 2019 to Nov 2019**.

## Question

1. Is it possible for some OEM that Assembly Date was not recorded when the product was manufactured but still it moves to the next stage ? What were the circumstances leading to those situation?
2. For some OEM this scenarios occued in late 2018 and for some this occured in late 2019. Does such scenarios differ based on some OEM companies policies  ?




# **3. Assembly Line Part 2 : Products that were assembled but were not shipped**

In [248]:
data_only_assembly_date  = data[ (data['Assembly Completed Date'].notna()) &   #Product is assembled
                              
                              (data['Date Shipped to Upfitter'].isna()) & 
                              ( data['Delivered to Dealer'].isna() ) &
                              ( data['Delivered to Client'].isna() ) &
                              ( data['Date Upfit Complete'].isna() ) &
                              ( data['Ship Date To Dealer'].isna()  ) ]


print("Products that were manufactured (Assembled) but have not been shipped for upfit or to dealer :", len(data_only_assembly_date))
data_only_assembly_date.head()

Products that were manufactured (Assembled) but have not been shipped for upfit or to dealer : 1266


,Identifier,Year,OEM,Model,Ordered Date,Assembly Completed Date,Date Shipped to Upfitter,Date Upfit Complete,Ship Date To Dealer,Delivered to Dealer,Delivered to Client
2997,ABCDEFGH123459947,2019,MAN E,Model M,2019-04-04,2019-08-09,NaT,NaT,NaT,NaT,NaT
8553,ABCDEFGH123465701,2019,MAN E,Model M,2018-10-18,2018-11-12,NaT,NaT,NaT,NaT,NaT
13765,ABCDEFGH123471043,2020,MAN C,Model B,2020-01-09,2020-02-06,NaT,NaT,NaT,NaT,NaT
13766,ABCDEFGH123471044,2020,MAN C,Model B,2020-01-09,2020-02-05,NaT,NaT,NaT,NaT,NaT
13767,ABCDEFGH123471045,2020,MAN C,Model B,2020-01-09,2020-02-05,NaT,NaT,NaT,NaT,NaT


In [249]:
data_only_assembly_date_c = data_only_assembly_date.iloc[:, 0:6].copy()
# print(data_only_assembly_date_c.head())
print(data_only_assembly_date_c.groupby('Assembly Completed Date').count())
fig = data_only_assembly_date_c.groupby('Assembly Completed Date').count()['Year'].plot(title="Overall Distribution of products that were manufactured but not proceeded further down the supply chain", 
                                                                       template="seaborn",
                                                                       labels=dict(index="Assembly Completed Date", 
                                                                                   value="Number of Parts",
                                                                                   variable="Option"
                                                                                   ))
fig.update_layout(showlegend=False)
fig.update_traces(mode="markers+lines")
fig.show()

                         Identifier  Year  OEM  Model  Ordered Date
Assembly Completed Date                                            
2018-11-12                        1     1    1      1             1
2019-05-17                        1     1    1      1             1
2019-07-29                        1     1    1      1             1
2019-08-09                        1     1    1      1             1
2019-08-12                        1     1    1      1             1
...                             ...   ...  ...    ...           ...
2020-06-04                       48    48   48     48            48
2020-06-05                       69    69   69     69            69
2020-06-06                       23    23   23     23            23
2020-06-07                        1     1    1      1             1
2020-06-08                       18    18   18     18            18

[91 rows x 5 columns]


In [250]:
df = data_only_assembly_date_c.groupby(['Assembly Completed Date', 'OEM'], as_index=False).count()
fig = px.line(df, x="Assembly Completed Date", y="Year", color="OEM",
              title="Parts Distribution based OEM that are only manufactured but not processed further ", 
              labels= dict(Year = 'Number of Parts'))
fig.update_traces(mode="markers+lines")
fig.update_layout(hovermode="x unified")

fig.show()


In [253]:
data_only_assembly_date_c_2020 = data_only_assembly_date_c[data_only_assembly_date_c['Assembly Completed Date'] >= '2020' ]
data_only_assembly_date_c_2020['Assembly_year_month'] = data_only_assembly_date_c_2020['Assembly Completed Date'].dt.to_period('M')

## Aggregating Data based on OEM , Model, Assembly Month

data_only_assembly_date_c_2020 = data_only_assembly_date_c_2020.groupby(['OEM','Model', 'Assembly_year_month'], as_index=False).count()[['OEM','Model','Assembly_year_month','Identifier' ]].copy()
data_only_assembly_date_c_2020.rename(columns= {'Identifier': 'Number of parts'}, inplace = True)
data_only_assembly_date_c_2020['Assembly_year_month'] = data_only_assembly_date_c_2020['Assembly_year_month'].astype(str)

#Plotting data
fig = px.line(data_only_assembly_date_c_2020, x="Assembly_year_month", y="Number of parts", color="Model", 
              facet_col = "OEM",
              title="Analysis of product that were Manufactured but not shipped further based on OEM and Models" 
              )
fig.update_traces(mode="markers+lines")
# fig.update_layout(hovermode="x unified")

fig.show()
# print(data_only_assembly_date_c_2020.sort_values(by = 'Number of parts', ascending=False))
print(data_only_assembly_date_c_2020.groupby(['OEM','Model'], 
                                      as_index=False).sum()[['OEM','Model','Number of parts']].sort_values(by = 'Number of parts', ascending=False))

     OEM                        Model  Number of parts
0  MAN B                     Model H               516
3  Man A  Model F COMMERCIAL STRIPPED              374
2  MAN C                      Model G              190
4  Man A                Model N CARGO              163
1  MAN C                      Model B               10
5  Man A              Model O CUTAWAY                1


## Observations

1. Total **1266** products are manufactured but were not shipped yet. Out of these, **1254** manufactured in **year 2020**
2. OEM - **Man B** and **Man A** has the highest products manufactured but were not processed further
3. Strange behaviour was recorded in OEM - **Man A** with Model - **Model F commercial Stripped** that has 180 products manufactured in Feb 2020 but were not processed further.
4. OEM - **Man B** with Model - **Model H** has **427** products manufactured in May 2020 but were not processed further.
5. **Major delay in manufactured product movement** all the OEM is recorded in **2020 only** , one reason could be COVID19.

## Questions

1. What is the average wait time for each OEM from manufacturing a product to ship for upfit or delvier to dealer?
2. Does Model F Commercial stripped was prioritised for shippment due to some reason(COVID19) 


# **4. Upfitment Line Part1 : Product that were delivered to dealer or client without upfittment**

In [254]:
data_not_upfit  = data[(data['Assembly Completed Date'].notna()) & 
                       
                       # No Upfit Date
                       ( data['Date Shipped to Upfitter'].isna() ) &
                       ( data['Date Upfit Complete'].isna() ) &

                       # Move to next stage
                       (
                       ( data['Delivered to Dealer'].notna() ) | 
                       ( data['Delivered to Client'].notna() ) |
                       ( data['Ship Date To Dealer'].notna() ) 
                       )  ]
print("Products that didn't come to upfitment stage but directly delivered further stages:", len(data_not_upfit))
data_not_upfit.head()

Products that didn't come to upfitment stage but directly delivered further stages: 6475


,Identifier,Year,OEM,Model,Ordered Date,Assembly Completed Date,Date Shipped to Upfitter,Date Upfit Complete,Ship Date To Dealer,Delivered to Dealer,Delivered to Client
2,ABCDEFGH123456791,2019,MAN E,Model M,2019-05-09,2019-09-26,NaT,NaT,2019-10-20,2019-10-21,NaT
6,ABCDEFGH123456795,2019,MAN E,Model M,2019-05-09,2019-09-23,NaT,NaT,2019-11-07,2019-11-11,NaT
10,ABCDEFGH123456799,2019,MAN E,Model M,2019-10-03,2019-09-23,NaT,NaT,2019-10-16,2019-10-18,NaT
21,ABCDEFGH123456812,2019,MAN E,Model M,2019-10-03,2019-09-19,NaT,NaT,2019-11-12,2019-12-03,2019-12-04
126,ABCDEFGH123456926,2019,MAN E,Model M,2019-05-09,2019-09-11,NaT,NaT,2019-10-20,2019-10-21,NaT


In [256]:
# data_not_upfit = data_not_upfit[data_not_upfit['Assembly Completed Date'] >= '2020' ]
data_not_upfit['Assembly_year_month'] = data_not_upfit['Assembly Completed Date'].dt.to_period('M')

## Aggregating Data based on OEM , Model, Assembly Month

data_not_upfit_c = data_not_upfit.groupby(['OEM','Model', 'Assembly_year_month'], as_index=False).count()[['OEM','Model','Assembly_year_month','Identifier' ]].copy()
data_not_upfit_c.rename(columns= {'Identifier': 'Number of parts'}, inplace = True)
data_not_upfit_c['Assembly_year_month'] = data_not_upfit_c['Assembly_year_month'].astype(str)

#Plotting data
fig = px.line(data_not_upfit_c, x="Assembly_year_month", y="Number of parts", color="Model", 
              facet_col = "OEM",
              title="Analysis of Products that didn't went through Upfitment stage based on OEM, Models, Assembly Date" 
              )
fig.update_traces(mode="markers+lines")
# fig.update_layout(hovermode="x unified")

fig.show()
# print(data_only_assembly_date_c_2020.sort_values(by = 'Number of parts', ascending=False))
print(data_not_upfit_c.groupby(['OEM','Model'], 
                                      as_index=False).sum()[['OEM','Model','Number of parts']].sort_values(by = 'Number of parts', ascending=False))

     OEM             Model  Number of parts
1  MAN E           Model M             4042
4  MAN F           Model K             2037
5  MAN F           Model L              366
2  MAN E     Model M CARGO               22
3  MAN F           Model D                5
6  Man A  Model E STRIPPED                2
0  MAN C           Model B                1


## Observations

1. Total **6475** products didn't went through uplift cycle and delivered to dealer or client.
2. OEM - **Man E** with **Model M** had the maximum miss Upliftment stage during **May to July 2018** with **3871** products.
3. OEM - **Man F** with **Model K** had the second highest miss upfitment products during **Aug to Sept 2018 and in October 2019** with **2036** products. 
4. OEM - **Man F** has missed most of the models upfitment stage in  **October 2019**

## Questions

1. Is it possible for products to skip upfitment stage? If yes, which OEM and models can skipt this stage ?
2. Does certain OEM has Upfitment stages shorter for different models in same OEM ? Is there any event in **Mid 2018 for OEM - MAN E** ?
 

# **5. Upfitment Line Part 2: Products that were not shipped but have upfittment completed**


In [257]:
data_not_ship_upfit  = data[ 
                       # No Upfit Date
                       ( data['Date Shipped to Upfitter'].isna() ) &
                       ( data['Date Upfit Complete'].notna() ) ]
print("Products that were not shipped fpr upfitment but had upfitment completed :", len(data_not_ship_upfit))
data_not_ship_upfit.head()

Products that were not shipped fpr upfitment but had upfitment completed : 23614


,Identifier,Year,OEM,Model,Ordered Date,Assembly Completed Date,Date Shipped to Upfitter,Date Upfit Complete,Ship Date To Dealer,Delivered to Dealer,Delivered to Client
0,ABCDEFGH123456789,2019,MAN E,Model M,2019-05-09,2019-09-26,NaT,2019-10-11,2019-10-23,2019-11-12,2019-11-12
1,ABCDEFGH123456790,2019,MAN E,Model M Cargo,2019-05-09,2019-09-29,NaT,2019-10-11,2019-10-15,2019-10-16,2019-11-06
3,ABCDEFGH123456792,2019,MAN E,Model M,2019-05-09,2019-09-25,NaT,2019-10-08,2019-11-01,2019-11-01,2019-11-18
4,ABCDEFGH123456793,2019,MAN E,Model M,2019-05-09,2019-09-29,NaT,2019-10-14,NaT,2019-11-21,2019-11-21
7,ABCDEFGH123456796,2019,MAN E,Model M,2019-05-09,2019-09-25,NaT,2019-10-01,2019-10-17,2020-02-25,2020-04-09


In [273]:
# data_not_upfit = data_not_upfit[data_not_upfit['Assembly Completed Date'] >= '2020' ]
data_not_ship_upfit['Upfit_date'] = data_not_ship_upfit['Date Upfit Complete'].dt.to_period('M')

## Aggregating Data based on OEM , Model, Assembly Month

data_not_ship_upfit_c = data_not_ship_upfit.groupby(['OEM','Model', 'Upfit_date'], as_index=False).count()[['OEM','Model','Upfit_date','Identifier' ]].copy()
data_not_ship_upfit_c.rename(columns= {'Identifier': 'Number of parts'}, inplace = True)
data_not_ship_upfit_c['Upfit_date'] = data_not_ship_upfit_c['Upfit_date'].astype(str)

#Plotting data
fig = px.line(data_not_ship_upfit_c, x="Upfit_date", y="Number of parts", color="Model", 
              facet_col = "OEM",
              title="Analysis of Products with no upfitment shipping date but have upfitment completed"
              )
fig.update_traces(mode="markers+lines")

fig.show()
# print(data_only_assembly_date_c_2020.sort_values(by = 'Number of parts', ascending=False))
print(data_not_ship_upfit_c.groupby(['OEM','Model'], 
                                      as_index=False).sum()[['OEM','Model','Number of parts']].sort_values(by = 'Number of parts', ascending=False))

      OEM                        Model  Number of parts
9   MAN F                      Model L            10865
6   MAN E                      Model M             3436
0   MAN B                     Model H              2743
11  Man A  Model F COMMERCIAL STRIPPED             2097
8   MAN F                      Model K             1822
10  Man A             Model E STRIPPED             1498
2   MAN C                      Model B              404
5   MAN D                      Model I              259
1   MAN C                      Model A              256
3   MAN C                      Model G              207
12  Man A                      Model N               24
4   MAN D                      Model A                1
7   MAN E                Model M Cargo                1
13  Man A                      Model O                1


## Observations

1. Total **23614** products didn't have uplift shipment date but had product uplift completed.
2. OEM - **Man E** with **Model M** had miss Upliftment shipment date during **July to October 2018** with around **3000** products.
3. OEM - **Man F** with **Model L** had the highest missing upfitment shipment date during **July 2019  to Feb 2020** with **9200** products. 


## Questions

1. Is it possible for products to skip upfitment shipment date? 
2. Is there a specific reason why most of the product in **Model L** are missing upfitment shipment date? It looks like **Model L** product has upfitted and manufactured at same place only.
3. Does OEM - **Man E** and **Man F** had some specific reason for skipping upfitment date ?   
 

In [ ]:
data_not_ship_deliver  = data[
                       ( data['Delivered to Dealer'].notna() ) &
                       ( data['Ship Date To Dealer'].isna() ) 
                        ]
print("Products that didn't have dealer shipment date but were delivered to dealer:", len(data_not_ship_deliver))
data_not_ship_deliver.head()

Products that didn't have dealer shipment date but were delivered to dealer: 9965


,Identifier,Year,OEM,Model,Ordered Date,Assembly Completed Date,Date Shipped to Upfitter,Date Upfit Complete,Ship Date To Dealer,Delivered to Dealer,Delivered to Client
4,ABCDEFGH123456793,2019,MAN E,Model M,2019-05-09,2019-09-29,NaT,2019-10-14,NaT,2019-11-21,2019-11-21
68,ABCDEFGH123456864,2019,MAN E,Model M,2019-04-04,2019-06-20,2019-08-27,2019-08-27,NaT,2019-09-13,2019-10-07
134,ABCDEFGH123456934,2019,MAN E,Model M,2019-04-04,2019-09-18,NaT,2019-10-11,NaT,2019-11-19,2019-11-19
162,ABCDEFGH123456962,2019,MAN E,Model M,2019-05-09,2019-09-24,NaT,2019-10-08,NaT,2019-12-20,2019-12-23
199,ABCDEFGH123456999,2019,MAN E,Model M,2019-03-18,2019-08-10,2019-09-30,2020-02-01,NaT,2019-10-07,2020-03-20


In [ ]:
# data_not_upfit = data_not_upfit[data_not_upfit['Assembly Completed Date'] >= '2020' ]
data_not_ship_deliver['delivered_date'] = data_not_ship_deliver['Date Upfit Complete'].dt.to_period('M')

## Aggregating Data based on OEM , Model, Assembly Month

data_not_ship_deliver_c = data_not_ship_deliver.groupby(['OEM','Model', 'delivered_date'], as_index=False).count()[['OEM','Model','delivered_date','Identifier' ]].copy()
data_not_ship_deliver_c.rename(columns= {'Identifier': 'Number of parts'}, inplace = True)
data_not_ship_deliver_c['delivered_date'] = data_not_ship_deliver_c['delivered_date'].astype(str)

#Plotting data
fig = px.line(data_not_ship_deliver_c, x="delivered_date", y="Number of parts", color="Model", 
              facet_col = "OEM",
              title="" 
              )
fig.update_traces(mode="markers+lines")
# fig.update_layout(hovermode="x unified")

fig.show()
# print(data_only_assembly_date_c_2020.sort_values(by = 'Number of parts', ascending=False))
print(data_not_ship_deliver_c.groupby(['OEM','Model'], 
                                      as_index=False).sum()[['OEM','Model','Number of parts']].sort_values(by = 'Number of parts', ascending=False))

      OEM                        Model  Number of parts
0   MAN B                     Model H              2642
9   Man A  Model F COMMERCIAL STRIPPED             1603
8   Man A             Model E STRIPPED             1498
7   MAN F                      Model L             1158
6   MAN F                      Model K              859
10  Man A                      Model N              648
4   MAN D                      Model I              187
2   MAN C                      Model B              173
1   MAN C                      Model A              157
5   MAN E                      Model M              111
3   MAN D                      Model A                1


## Observations

1. Total **9965** products didn't have delivery shipment date but had been delivered to Dealer.
2. OEM - **Man B** with **Model H** don't have dealer shipment date **Dec 2019, Feb 2019, April 2019** with around **2642** products.
3. OEM - **Man A** with **Model F COMMERCIAL STRIPPED** & **Model E STRIPPED** and had the highest missing dealer shipment date during **July 2019  to Feb 2020** with **3000** products. 


## Questions

1. Why certain OEM don't have dealer shipment date but are delivered to dealer? 
2. Does OEM - **Man A** and **Man B** had some specific reason for skipping  dealer shipment date during mid 2019? 
 

# **6. Delivery Line: Products which are directy delivered to client skipping dealer dates**

In [274]:
data_no_dealar  = data[
                       ( (data['Delivered to Dealer'].isna() ) | (data['Ship Date To Dealer'].isna()) ) &
                       ( data['Delivered to Client'].notna() ) 
                        ]
print("Products that didn't have any dealer delivery or dealer shipment date but delivered to client directly:", len(data_no_dealar))
data_no_dealar.head()

Products that didn't have any dealer delivery or dealer shipment date but delivered to client directly: 9492


,Identifier,Year,OEM,Model,Ordered Date,Assembly Completed Date,Date Shipped to Upfitter,Date Upfit Complete,Ship Date To Dealer,Delivered to Dealer,Delivered to Client
4,ABCDEFGH123456793,2019,MAN E,Model M,2019-05-09,2019-09-29,NaT,2019-10-14,NaT,2019-11-21,2019-11-21
68,ABCDEFGH123456864,2019,MAN E,Model M,2019-04-04,2019-06-20,2019-08-27,2019-08-27,NaT,2019-09-13,2019-10-07
134,ABCDEFGH123456934,2019,MAN E,Model M,2019-04-04,2019-09-18,NaT,2019-10-11,NaT,2019-11-19,2019-11-19
162,ABCDEFGH123456962,2019,MAN E,Model M,2019-05-09,2019-09-24,NaT,2019-10-08,NaT,2019-12-20,2019-12-23
199,ABCDEFGH123456999,2019,MAN E,Model M,2019-03-18,2019-08-10,2019-09-30,2020-02-01,NaT,2019-10-07,2020-03-20


In [276]:
# data_not_upfit = data_not_upfit[data_not_upfit['Assembly Completed Date'] >= '2020' ]
data_no_dealar['delivered_date'] = data_no_dealar['Delivered to Client'].dt.to_period('M')

## Aggregating Data based on OEM , Model, Assembly Month

data_no_dealar_c = data_no_dealar.groupby(['OEM','Model', 'delivered_date'], as_index=False).count()[['OEM','Model','delivered_date','Identifier' ]].copy()
data_no_dealar_c.rename(columns= {'Identifier': 'Number of parts'}, inplace = True)
data_no_dealar_c['delivered_date'] = data_not_ship_deliver_c['delivered_date'].astype(str)

#Plotting data
fig = px.line(data_no_dealar_c, x="delivered_date", y="Number of parts", color="Model", 
              facet_col = "OEM",
              title="Analysis for products missing dealer interactions" 
              )
fig.update_traces(mode="markers+lines")
# fig.update_layout(hovermode="x unified")

fig.show()
# print(data_only_assembly_date_c_2020.sort_values(by = 'Number of parts', ascending=False))
print(data_no_dealar_c.groupby(['OEM','Model'], 
                                      as_index=False).sum()[['OEM','Model','Number of parts']].sort_values(by = 'Number of parts', ascending=False))

      OEM                        Model  Number of parts
5   MAN E                      Model M             4068
8   MAN F                      Model K             2736
9   MAN F                      Model L             1494
12  Man A                      Model N              676
1   MAN C                      Model A              141
2   MAN C                      Model B              134
4   MAN D                      Model I               85
10  Man A             Model E STRIPPED               81
11  Man A  Model F COMMERCIAL STRIPPED               44
6   MAN E                Model M CARGO               22
7   MAN F                      Model D                5
0   MAN B                     Model H                 4
3   MAN D                      Model A                1
13  Man A                      Model O                1


## Observations

1. Total **9492** products that wre deliverd to client directly withour dealer.
2. OEM - **Man E** with **Model M** don't have dealer involved in the insurance **March 2019 to July 2019** with around **4068** products.
3. OEM - **Man F** with **Model K** and had the highest missing dealer date with **2736** products. 


## Questions

1. Why OEM - **Man E** don't have any dealer dates but were directly delivered to client? 
2. Is there a specific startegy followed by **OEM - MAN E** for product deliver ?
 

---
# Overall Observations
---

## 1. Ordering Line :

It is evident that there is some order issue with **Original Equipment Manufacture (OEM)** - **Man A** and **MAN F** as they have highest products that were ordered but not shipped to dealer or client.

1. Man A : **3206 Parts**
2. MAN F : **525 Parts**

## 2. Assembly Line Part 1 :

1. Total 88 Products were found with no assembly date but moved to further stages in the process.
2. Products with **OEM**  - **Man E** and **Man A** have maximum products that don't have assembly date but were delivered to dealer or client.
3. OEM with **Man E** had something wrong in **3rd Quater of 2018** as they have maximum products that don't have assembly date during that time with **44 products**
4. OEM with **Man A** has missing assembly issue around **Aug 2019 to Nov 2019** with **30 Product**.


## 3. Assembly Line Part 2 :

1. Total **1266** products are manufactured but were not shipped yet. Out of these, **1254** manufactured in **year 2020**
2. OEM - **Man B** and **Man A** has the highest products manufactured but were not processed further
3. Strange behaviour was recorded in OEM - **Man A** with Model - **Model F commercial Stripped** that has 180 products manufactured in Feb 2020 but were not processed further.
4. OEM - **Man B** with Model - **Model H** has **427** products manufactured in May 2020 but were not processed further.
5. **Major delay in manufactured product movement** all the OEM is recorded in **2020 only** , one reason could be COVID19.

## 4. Upfitment Line Part 1 : 
## Observations

1. Total **6475** products didn't went through uplift cycle and delivered to dealer or client.
2. OEM - **Man E** with **Model M** had the maximum miss Upliftment stage during **May to July 2018** with **3871** products.
3. OEM - **Man F** with **Model K** had the second highest miss upfitment products during **Aug to Sept 2018 and in October 2019** with **2036** products. 
4. OEM - **Man F** has missed most of the models upfitment stage in  **October 2019**
 

## 5. Upfitment Line Part 2 : 

1. Total **23614** products didn't have uplift shipment date but had product uplift completed.
2. OEM - **Man E** with **Model M** had miss Upliftment shipment date during **July to October 2018** with around **3000** products.
3. OEM - **Man F** with **Model L** had the highest missing upfitment shipment date during **July 2019  to Feb 2020** with **9200** products. 
 
 

## 6. Delivery Line : 

1. Total **9492** products that wre deliverd to client directly withour dealer.
2. OEM - **Man E** with **Model M** don't have dealer involved in the insurance **March 2019 to July 2019** with around **4068** products.
3. OEM - **Man F** with **Model K** and had the highest missing dealer date with **2736** products. 




---
# Overall Questions


---


## 1. Ordering Line :

What is the order collection process for OEM - Man F and Man A? How long is the time taken to process OEM parts to further stages? 

## 2. Assembly Line Part 1 :

1. Is it possible for some OEM that Assembly Date was not recorded when the product was manufactured but still it moves to the next stage ? What were the circumstances leading to those situation?
2. For some OEM this scenarios occued in late 2018 and for some this occured in late 2019. Does such scenarios differ based on some OEM companies policies  ?



## 3. Assembly Line Part 2 :


1. What is the average wait time for each OEM from manufacturing a product to ship for upfit or delvier to dealer?
2. Does Model F Commercial stripped was prioritised for shippment due to some reason(COVID19) 


## 4. Upfitment Line Part 1 : 

1. Is it possible for products to skip upfitment stage? If yes, which OEM and models can skipt this stage ?
2. Does certain OEM has Upfitment stages shorter for different models in same OEM ? Is there any event in **Mid 2018 for OEM - MAN E** ?

## 5. Upfitment Line Part 2 : 

1. Is it possible for products to skip upfitment shipment date? 
2. Is there a specific reason why most of the product in **Model L** are missing upfitment shipment date? It looks like **Model L** product has upfitted and manufactured at same place only.
3. Does OEM - **Man E** and **Man F** had some specific reason for skipping upfitment date ?  

## 6. Delivery Line : 

1. Why OEM - **Man E** don't have any dealer dates but were directly delivered to client? 
2. Is there a specific startegy followed by **OEM - MAN E** for product deliver ?
 


--- needs to be checked ---

## 1. Some of the items are missing shipped date to **upfitter**. Is the part manufactured and upfit at same place? If not what are the scenarios where the date to upfit is not given 

## 2. For some Assembly completed date is not available but the product is delivered to client/ dealer - **ABCDEFGH123459158**. Is it the normal scenario ?

## 3.

